In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense,Input,Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import SGD,Adam
import pandas as pd


In [ ]:
df = pd.read_csv("/content/train.csv")
df['Sentence'] = df['Sentence'].str.lower()

In [ ]:
df.head()

,Id,Sentence,Entity_1,Entity_2,Relation
0,1,rising interest rates affects borrowing costs ...,Interest Rates,borrowers,affects
1,3,market volatility affects traders by leading t...,Market Volatility,Traders,affects
2,4,strong corporate earnings affect shareholders ...,Corporate Earnings,Shareholders,affects
3,5,regulatory changes affect financial institutio...,Regulatory Changes,Financial Institutions,affects
4,6,investor behavior affects market trends and as...,Investor Behavior,Market Trends,affects


In [ ]:

sentences=[]
for jok in df['Sentence']:
    sentences.append(jok)
word1=[]
for jok in df['Entity_1']:
    word1.append(jok)
word2=[]
for jok in df['Entity_2']:
    word2.append(jok)
relations=[]
for jok in df['Relation']:
    relations.append(jok)

# Separate data into input (sentences) and output (relations)
#sentences = np.array(sentences)
#word1 = np.array([df['Entity_1']])
#word2 = np.array([df['Entity_2']])
#relations = np.array([df['Relation']])
#sentences=sentences.flatten()
#word1=word1.flatten()
#word2=word2.flatten()
#relations=relations.flatten()
#print(relations)
# Tokenize sentences



In [ ]:
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils
import numpy as np
from transformers import TFBertForSequenceClassification, BertTokenizer
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


# Load pre-trained BERT model and tokenizer
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=12)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Set maximum sequence length
max_length = 128  # You can adjust this value as needed

# Tokenize input sentences
tokenized_inputs = tokenizer(sentences, truncation=True, padding=True, max_length=max_length, return_tensors='tf')

# Convert tokenized inputs to numpy arrays
X = np.array(tokenized_inputs['input_ids'])
token_type_ids = np.array(tokenized_inputs['token_type_ids'])
attention_masks = np.array(tokenized_inputs['attention_mask'])

# Convert labels to numpy arrays
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(relations)

# Split data into train and test sets
X_train, X_test, token_type_ids_train, token_type_ids_test, attention_masks_train, attention_masks_test, y_train, y_test = train_test_split(X, token_type_ids, attention_masks, y, test_size=0.2, random_state=42)

import torch.optim as optim

# Create an instance of the Adam optimizer
optimizer = optim.Adam(parameters, lr=learning_rate, betas=(beta1, beta2), eps=epsilon, weight_decay=weight_decay, amsgrad=amsgrad)


# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit([X_train, token_type_ids_train, attention_masks_train], y_train, validation_data=([X_test, token_type_ids_test, attention_masks_test], y_test), epochs=3, batch_size=32)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'parameters' is not defined

In [ ]:
import torch
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset

# Load pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=12)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input sentences and encode labels
tokenized_inputs = tokenizer(sentences, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
X = tokenized_inputs['input_ids']
token_type_ids = tokenized_inputs['token_type_ids']
attention_masks = tokenized_inputs['attention_mask']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(relations)

# Split data into train and test sets
X_train, X_test, token_type_ids_train, token_type_ids_test, attention_masks_train, attention_masks_test, y_train, y_test = train_test_split(X, token_type_ids, attention_masks, y, test_size=0.2, random_state=42)

# Convert data to PyTorch DataLoader
train_dataset = TensorDataset(X_train, token_type_ids_train, attention_masks_train, torch.tensor(y_train))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test, token_type_ids_test, attention_masks_test, torch.tensor(y_test))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the optimizer
learning_rate = 2e-5
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8
weight_decay = 0.0
amsgrad = False
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(beta1, beta2), eps=epsilon, weight_decay=weight_decay, amsgrad=amsgrad)

# Define loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, token_type_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, token_type_ids, attention_mask, labels = batch
            outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs.logits, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = total_correct / total_samples
    print(f'Epoch {epoch + 1}/{num_epochs}, Accuracy: {accuracy}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
